In [363]:
import transformers
import torch
import pandas as pd
import numpy as np
import json
from pathlib import Path

import spacy
import nltk
nltk.download('stopwords')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

import scipy.stats as st

[nltk_data] Downloading package stopwords to /Users/Sanna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# containing descriptions of columns
with open('meta_dict.json', 'r') as f: # descriptions of columns
    meta_dict = json.load(f)
print(len(meta_dict))

52


In [483]:
ROOT = Path('/Volumes/INWT/Daten_NLP/') # encrypted folder!
DATA = ROOT / '200707_aachener_zeitung_modified.csv' # text is already minimal preprocessed

In [484]:
df = pd.read_csv(DATA, index_col = 0)
df.head()

,pageviews,entrances,exits,bounces,timeOnPage,conversions,avgTimeOnPage,stickiness,entranceRate,bounceRate,...,text,titelH1,titelH3,wordcount,category,city,text_preprocessed,nr_tokens,mean_token_length,avgTimeOnPage/wordcount
articleId,,,,,,,,,,,,,,,,,,,,,
48620281,21,7,12,7,1012,NaN,112.444444,42.857143,33.333333,33.333333,...,München/Stuttgart (dpa/tmn) - Frische Luft und...,Ohne Treter läuft nichts: Die richtigen Schuhe...,NaN,769,vm,München/Stuttgart,Frische Luft und Bewegung: Diese Kombination r...,796,5.359296,0.146222
48620381,19,6,11,5,1484,NaN,185.500000,42.105263,31.578947,26.315789,...,Berlin/Frankfurt/Main (dpa/tmn) - Der Wecker k...,Ohne Antrieb: Wann ist der Winter-Blues eine D...,NaN,441,vm,Berlin/Frankfurt/Main,"Der Wecker klingelt, aufstehen! Doch gerade im...",452,5.938053,0.420635
48622639,2,2,2,2,0,NaN,0.000000,0.000000,100.000000,100.000000,...,Berlin (dpa/tmn) - Eltern auf der Suche nach e...,Funk statt WLAN: Klassische Babyphones weiter ...,NaN,390,vm,Berlin,Eltern auf der Suche nach einem guten Babyphon...,396,5.848485,0.000000
48623085,32,9,20,9,974,NaN,81.166667,37.500000,28.125000,28.125000,...,Berlin (dpa/tmn) - Spülmaschinentabs sollen kl...,Warentest: Die besten Spülmaschinen-Multitabs ...,NaN,345,vm,Berlin,Spülmaschinentabs sollen kleine Alleskönner se...,367,5.594005,0.235266
48623259,24,2,7,2,3797,NaN,223.352941,70.833333,8.333333,8.333333,...,Berlin (dpa/tmn) - Make-up hat heutzutage eine...,Ist das noch natürlich? - Stiftung Warentest p...,NaN,182,vm,Berlin,Make-up hat heutzutage einen Zweck: Es soll da...,183,5.622951,1.227214


In [485]:
df.shape

(891, 33)

In [486]:
df.loc[49200513]

pageviews                                                               3047
entrances                                                               1119
exits                                                                   1693
bounces                                                                  924
timeOnPage                                                            217337
conversions                                                              NaN
avgTimeOnPage                                                        160.515
stickiness                                                           44.4372
entranceRate                                                         36.7246
bounceRate                                                           30.3249
kategorie                                                           ratgeber
rubrik                                                                 recht
subRubrik                                                                NaN

In [487]:
df.loc[49200513, 'text_preprocessed'] # one example

'Die neuen Fälle von Ansteckungen mit dem Coronavirus Sars-CoV-2 haben zum Teil Konsequenzen für öffentliche Einrichtungen. In Nordrhein-Westfalen sind als Vorsichtsmaßnahme gegen die Krankheit Covid-19 einzelne Kindergärten und Schulen geschlossen, auf Teneriffa sitzen Urlauber in einem Hotel fest. Was gilt für Arbeitgeber in solchen Situationen - und was sollten Beschäftigte noch dazu wissen? Fall 1: Die Kita oder Schule bleibt geschlossen Wenn Kindergärten und Schulen aus Vorsicht zum Infektionsschutz geschlossen sind, können Arbeitnehmer im Notfall zu Hause bleiben, um ihre Kinder zu betreuen, erklärt Nathalie Oberthür, Fachanwältin für Arbeitsrecht aus Köln. Ob sie dann auch weiter ihr Gehalt bekommen, hängt aber davon ab, ob wirklich keine andere Betreuung möglich war. Paragraf 616 im Bürgerlichen Gesetzbuch (BGB) besagt, dass, wer ohne eigenes Verschulden und aus einem persönlichen Grund verhindert ist und nicht zur Arbeit kommen kann, trotzdem weiter Gehalt bekommt. Die erste M

In [491]:
df = df.fillna('') # replacing Nan with emtpy string

In [492]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 48620281 to 51565349
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   pageviews                891 non-null    int64  
 1   entrances                891 non-null    int64  
 2   exits                    891 non-null    int64  
 3   bounces                  891 non-null    int64  
 4   timeOnPage               891 non-null    int64  
 5   conversions              891 non-null    object 
 6   avgTimeOnPage            891 non-null    float64
 7   stickiness               891 non-null    float64
 8   entranceRate             891 non-null    float64
 9   bounceRate               891 non-null    float64
 10  kategorie                891 non-null    object 
 11  rubrik                   891 non-null    object 
 12  subRubrik                891 non-null    object 
 13  titel                    891 non-null    object 
 14  pagePath      

In [514]:
#df.sort_values(by='pageviews', ascending=False)
df.sort_values(by='timeOnPage', ascending=False)
#df.sort_values(by='avgTimeOnPage', ascending=False)
#df.sort_values(by='stickiness', ascending=False)

,pageviews,entrances,exits,bounces,timeOnPage,conversions,avgTimeOnPage,stickiness,entranceRate,bounceRate,...,text,titelH1,titelH3,wordcount,category,city,text_preprocessed,nr_tokens,mean_token_length,avgTimeOnPage/wordcount
articleId,,,,,,,,,,,,,,,,,,,,,
49200513,3047,1119,1693,924,217337,,160.514771,44.437151,36.724647,30.324910,...,Köln (dpa/tmn) - Die neuen Fälle von Ansteckun...,Einschränkungen wegen Covid-19: Was Arbeitnehm...,,376,vm,Köln,Die neuen Fälle von Ansteckungen mit dem Coron...,379,5.598945,0.426901
49473155,1887,712,1006,587,169090,,191.929625,46.687864,37.731849,31.107578,...,Düsseldorf (dpa/tmn) - Um die Verbreitung des ...,Veranstaltung wegen Corona abgesagt: Rechte de...,,708,vm,Düsseldorf,Um die Verbreitung des neuen Coronavirus Sars-...,718,5.988858,0.271087
49333693,2487,1877,2032,1699,136927,,300.938462,18.295135,75.472457,68.315239,...,Frankfurt/Main/Düsseldorf (dpa/tmn) - Nachdem ...,Covid-19: Welche Regeln gelten im Betrieb?,,768,vm,Frankfurt/Main/Düsseldorf,Nachdem mehr und mehr Fälle von Covid-19-Infek...,770,6.137662,0.391847
49321583,1709,1226,1333,1085,92555,,246.156915,22.001170,71.737858,63.487420,...,Berlin (dpa/tmn) - Der neuartige Coronavirus S...,Ansteckung und Vorbeugung: Der Alltag mit Covi...,,882,vm,Berlin,Der neuartige Coronavirus Sars-Cov-2 verbreite...,889,5.823397,0.279089
49232551,1548,580,811,468,77052,,104.548168,47.609819,37.467700,30.232558,...,Berlin (dpa/tmn) - Eine Impfung gegen das neue...,Coronavirus: Experten raten zu Impfungen - auc...,,253,vm,Berlin,Eine Impfung gegen das neue Coronavirus Sars-C...,252,6.178571,0.413234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48870125,2,2,2,2,0,,0.000000,0.000000,100.000000,100.000000,...,Frankfurt/Main (dpa/tmn) - Wonach sehnen Sie s...,Sinnbild für Exotik: Der Affe im Dschungel als...,,415,vm,Frankfurt/Main,Wonach sehnen Sie sich? Den Wald im regnerisch...,427,5.341920,0.000000
49133249,2,2,2,2,0,,0.000000,0.000000,100.000000,100.000000,...,Berlin (dpa/tmn) - Wenn die Schönheitsreparatu...,Schönheitsreparaturen auch in möblierten Wohnu...,,148,vm,Berlin,Wenn die Schönheitsreparaturen in einer Wohnun...,143,6.265734,0.000000
49571165,21,21,21,21,0,,0.000000,0.000000,100.000000,100.000000,...,Cottbus/Berlin (dpa/tmn) - Ein Job am Schreibt...,Bei Wind und Wetter im Einsatz: Die Ausbildung...,,638,vm,Cottbus/Berlin,"Ein Job am Schreibtisch, das wäre nichts für A...",650,5.755385,0.000000


## Modeling pageviews / avgTimeOnPage with (lemmatized) Bag-of-Words (Logistic Regression)

In [461]:
#creating train, dev, test
RANDOM_SEED = 123
df_train, df_test = train_test_split(df, test_size=0.2, random_state=RANDOM_SEED, shuffle=True)
df_dev, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED, shuffle=True)
print(df_train.shape, df_dev.shape, df_test.shape)

(712, 33) (89, 33) (90, 33)


In [462]:
# lemmatizes (punctuation stays in, but the default tokenizer from Count_Vectorizer seems to delete it)
class Preprocessor():
    def __init__(self):
        self.nlp = spacy.load("de_core_news_sm", disable=['parser', 'ner'])
        #self.nlp = spacy.load("de_core_news_md", disable=['parser', 'ner'])

    def __call__(self, doc):
        rt = []
        doc = self.nlp(doc)
        for token in doc:
            rt.append(token.lemma_.lower()) # stopwords stay in, but CountVectorizer uses nltk-list later

        return " ".join(rt)

In [463]:
stopwords = nltk.corpus.stopwords.words('german')
print(stopwords[:10])
print(len(stopwords))
stopwords = [ preprocessor(s) for s in stopwords ]
print(stopwords[:10])
print(len(stopwords))

['aber', 'alle', 'allem', 'allen', 'aller', 'alles', 'als', 'also', 'am', 'an']
232
['aber', 'all', 'alle', 'alle', 'all', 'alle', 'als', 'also', 'am', 'an']
232


In [464]:
preprocessor = Preprocessor()
vectorizer = CountVectorizer(analyzer='word',
                             preprocessor=preprocessor,
                             ngram_range=(1, 5),
                             min_df=5,
                             max_features=10000,
                             stop_words=stopwords
                            )

In [494]:
# define text for features: text? text_preprocessed? titelH1?

feature = 'text_preprocessed'
#feature = 'text'
#feature = 'titelH1'
#feature = 'teaser' # note: not all have 'teaser' but I replaced Nan wih empty string ""

X_train = vectorizer.fit_transform(df_train[feature])

In [497]:
vectorizer.get_feature_names()[100:120] # just some n-grams

['30 sekunde',
 '300',
 '3000',
 '31',
 '35',
 '350',
 '37',
 '39',
 '40',
 '40 000',
 '400',
 '42',
 '43',
 '45',
 '46',
 '49',
 '50',
 '50 euro',
 '50 meter',
 '50 prozent']

In [498]:
X_dev = vectorizer.transform(df_dev[feature])
X_test = vectorizer.transform(df_test[feature])

In [499]:
X_train, X_dev, X_test = X_train.toarray(), X_dev.toarray(), X_test.toarray()

In [500]:
X_train.shape, X_dev.shape, X_test.shape

((712, 4580), (89, 4580), (90, 4580))

In [515]:
# define the target labels
target = 'timeOnPage'
#target =  'pageviews'
#target = 'avgTimeOnPage'
#target = 'stickiness'

y_train = np.array(df_train[target])
y_dev = np.array(df_dev[target])
y_test = np.array(df_test[target])

In [516]:
y_train.shape, y_dev.shape, y_test.shape

((712,), (89,), (90,))

In [517]:
from sklearn.linear_model import LogisticRegression, Ridge, LinearRegression, SGDRegressor

In [518]:
# choose model ???
#model = LogisticRegression(max_iter=10000)
model = Ridge()
#model = LinearRegression()
#model = SGDRegressor()

In [519]:
model.fit(X_train, y_train)

Ridge()

In [520]:
# predict for dev set
pred_dev = model.predict(X_dev)

In [521]:
pred_dev

array([ 4.20962959e+04,  5.61771686e+02,  6.18724779e+03,  2.24673152e+03,
       -3.78684791e+02,  3.37787517e+03, -1.74386909e+03,  1.13827935e+03,
        1.22911070e+03, -1.30433197e+03,  3.25023580e+03, -3.70038085e+03,
       -1.73331072e+03,  1.64354535e+03,  3.87576193e+03,  2.48131840e+02,
        1.29126878e+03,  1.25634221e+03,  4.33253027e+01,  1.25380374e+03,
        6.29259682e+03, -1.79649042e+03,  4.52123175e+03, -2.46520311e+03,
        1.80833105e+03,  6.02626270e+02, -3.46888282e+03, -2.28633961e+03,
       -3.16622658e+03,  4.98376299e+03,  1.60032694e+03,  7.50229834e+03,
        2.68606716e+03,  1.43716732e+04, -3.76047759e+03,  6.34330655e+02,
       -1.15182628e+03,  3.98835168e+03,  6.89867156e+03, -1.72014262e+03,
        3.70031948e+03,  6.31788799e+02,  2.14121286e+03,  4.92033019e+03,
        2.65979123e+03, -4.73622211e+03, -6.37539790e+02,  1.87571912e+04,
        4.79583692e+03, -2.42894575e+03,  1.10492588e+04, -9.17251297e+03,
        1.28195328e+03,  

In [522]:
np.array(y_dev)

array([217337,    912,    428,    116,      0,   1133,   1490,   1393,
           78,    235,      0,    948,     56,  20987,     90,    173,
          255,  10379,    907,   8279,    240,    399,   2225,    427,
         1503,    951,    679,    239,   1003,     30,    204,    643,
          799,   4095,   1049,    373,   2207,    133,    794,    265,
          272,     29,    108,    278,   1193,     62,     83,   2224,
          209,    585,    257,   1786,    724,   2302,   1240,    264,
          714,   2666,    247,   3936,    490,    950,      0,   1860,
         1018,     98,    749,    267,   1444,    702,     68,      0,
           99,     60,    691,    442,   1995,    390,   6884,    774,
            0,    761,   1508,   3243,      0,   1148,     31,   1399,
         1484])

In [523]:
st.pearsonr(pred_dev, y_dev)

(0.6770499825890346, 3.234888149141273e-13)

In [524]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score

In [525]:
mean_squared_error(pred_dev, y_dev) #, squared=False) # hard to interpret

371739874.8163862

In [526]:
mean_absolute_error(pred_dev, y_dev) # hard to interpret

5716.443798718035

In [527]:
explained_variance_score(pred_dev, y_dev) # what does this number mean? ... (1.0 is best)

-8.380319763788506

In [398]:
df_dev.iloc[4] # Example where timeOnPage = 0 (boring article! :D)

pageviews                                                                  1
entrances                                                                  0
exits                                                                      1
bounces                                                                    0
timeOnPage                                                                 0
conversions                                                              NaN
avgTimeOnPage                                                              0
stickiness                                                                 0
entranceRate                                                               0
bounceRate                                                                 0
kategorie                                                           ratgeber
rubrik                                                                reisen
subRubrik                                                                NaN

In [192]:
#vectorizer.stop_words_ # shows not used features (in too many/little documents)
#vectorizer.vocabulary_ # values are the indices! (not frequency or anything)

## Beobachtung

Bei timeOnPage ist das Modell sehr gut, bei pageviews allerings nicht so sehr. Woran liegt das? Lernt das Modell vielleicht eher anhand der reinen Textlänge und nicht spezifische Wörter? Hier wäre vielleicht Shap gut...

Bei pageviews vielleicht eher den Titel oder Teaser als den Text selbst nehmen? (--> etwas besser, dann aber min_df ändern)

Welches Label ist das sinnvollste?

Welche Metrik? Pearson Korrelation? Absolute Error?

Vielleicht mal einfach die Keywords, Subjekts aufsplitten und als Features nehmen?